In [ ]:
# file: seeq/models/spinboson.py
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import LinearOperator
from seeq.models.lattice import Lattice, Regular3DLattice

# Spin boson model

## The problem

The spin-boson model is a two-level system coupled to a bosonic environment
$$ H = \frac{\Delta}{2}\sigma^z + \sum_{jk} \bar{H}_{kj} a^\dagger_k a_j + O \sum_k (g_k a^\dagger_k + \mathrm{H.c.}).$$
The bosonic modes are subject to a linear Hamiltonian, represented by the coupling matrix $\bar{H},$ which will be usually created using a `Lattice` object (see notebook [Lattices](Lattices.ipynb)). These modes couple to the quantum emitter through coupling strengths $\{g_k\}.$

### a) Polaron Hamiltonian

This problem is usually approximated by the Rotating-Wave Approximation (RWA) model
$$ H_{RWA} = \frac{\Delta}{2}\sigma^z + \sum_{kj} H_{kj} a^\dagger_k a_j+ \sum_k (g_k \sigma^- a^\dagger_k + \mathrm{H.c.}),$$
which neglects the counter-rotating terms $\sigma^+ a_k^\dagger$ and $\sigma^- a_k.$

Instead of doing this, we massage the full Hamiltonian to obtain new couplings using the Silbey-Harris theory. As explained in [T. Shi et al PRL 120, 153602 (2018)](https://doi.org/10.1103/PhysRevLett.120.153602), the effective model based on the polaron Hamiltonian takes the following form
$$H=\frac{\tilde{\Delta}}{2}\sigma^z e^{-2\sigma^x B^\dagger}e^{2\sigma^x B} + \sum_{kj} H_{kj} a^\dagger_k a_j + \tilde{\Delta}\sigma^x (B + B^\dagger)$$
with the collective modes
$$B=\sum_k f_k^* a_k.$$

The displacements $f_k$ can be grouped into a vector $\vec{f}$ that satisfies the following self-consistent relations
$$\vec{f} = \frac{1}{\bar{H} + \tilde{\Delta}} \vec{g}$$
followed by
$$\tilde\Delta = \Delta \exp\left(-2 \vec{f}^\dagger \vec{f}\right).$$

In [ ]:
# file: seeq/models/spinboson.py

def SilbeyHarris(H, g, Δ=1.0):
    """Compute the effective couplings and renormalized frequency
    of the Silbey-Harris transformation, using the self-consistent
    equations.
    
    Parameters
    ----------
    H  -- Hamiltonian of the bosons (matrix or LinearOperator)
    g  -- Vector of couplings between the emitter and the bosons
    Δ  -- Gap of the quantum emitter (defaults to 1)
    
    Returns
    -------
    f  -- Vector of displacements
    Δr -- Renormalized gap for the emitter
    """
    Δr = Δ
    f = g
    if np.sum(np.abs(g)) > 1e-15:
        #
        # The Silbey-Harris procedure only works when the Hamiltonian
        # of the bosons has a nonzero gap.
        λ = sp.linalg.eigsh(H, k=1, which='SA', return_eigenvectors=False)
        if np.min(λ) <= 0:
            raise Exception(f'Eigenvalue negative or zero {λ}, singular lattice.')
        #
        # Solve the coupled equations iteratively.
        Id = sp.eye(env.size)
        for i in range(20):
            newf = sp.linalg.spsolve(env.H + Δr * Id, g)
            newΔr = Δ * np.exp(-2*np.vdot(newf,newf).real)
            if f.size > 1:
                err = 1.0 - np.vdot(newf,f)/(np.linalg.norm(f)*np.linalg.norm(newf))
            else:
                err = np.linalg.norm(newf-f)/np.linalg.norm(f)
            f = newf
            Δr = newΔr
            if abs(err) < 1e-12:
                break
    return f, Δr

### b) Chain mapping

We can convert any of our environment models from a spin boson formulation with a generic number of modes
$$H = \frac{\Delta}{2}\sigma^z + \sum_k g_k \sigma^x(a_k + a_k^\dagger) + \sum_k \omega_k a_k^\dagger a_k$$
to a tight-binding model
$$H = \frac{\Delta}{2}\sigma^z + \beta_0 \sigma^x (b_0+b_0^\dagger) + \sum_{k=0}^{N-1} (\beta_k b_k^\dagger b_{k+1}+\mathrm{H.c.}) + \sum_k \alpha_k b_k^\dagger b_k$$

In [ ]:
class LanczosEnv(Lattice):
    def __init__(self, lattice, g, tol=1e-10, nmodes=None, debug=False):
        """Construct an environment for an 1D model, up fom a general environment in
        a higher-dimensional space.

        Arguments:
        ----------
        lattice  -- Original problem
        g        -- Vector of couplings from the qubit to the local modes
        tol      -- Limit for stopping the Lanczos recursion
                    (defaults to 1e-10)
        nmodes   -- Maximum number of modes for the expansion
                    (defaults to all modes)
        """
        H = lattice
        x = g
        L = H.shape[0]
        if nmodes is None:
            nmodes = L
        beta0 = np.linalg.norm(x)
        x /= beta0
        # The rows of 'w' are the modes 'b_k' of the new model
        w = [x]
        for i in range(nmodes-1):
            x = H @ x
            for y in w:
                x = x - np.dot(y, x) * y
            nrm = np.linalg.norm(x)
            if nrm <= tol:
                break
            x /= nrm
            w = [x] + w
        # The columns of 'w' are the modes 'b_k' of the new model
        w = np.array(list(reversed(w))).T
        # We diagonalize the Hamiltonian on the projected subspace
        H = w.T.conjugate() @ (H @ w)
        # We extract the only relevant items, neglecting small values
        H = sum(sp.diags([np.diag(H, i)], [i]) for i in [-1,0,1])
        g = np.zeros(w.shape[1])
        g[0] = 1.0
        super(LanczosEnv, self).__init__(H, dimension=1)
        self.basis = w
        self.g = np.zeros(w.shape[1])
        self.g[0] = 1.0
        self.orig_lattice = lattice
        print(f'Lanczos basis {w.shape}')

### c) Single-photon polaron

In the limit of single excitations, we can approximate
$$e^{-2\sigma^x B^\dagger}e^{2\sigma^x B} \simeq 1 + 2\sigma^x (B - B^\dagger) - 4 B^\dagger B.$$
Using $\sigma^z\sigma^x = i\sigma^y$ we obtain
$$H_{polRWA} = \frac{\tilde\Delta}{2}\sigma^z - 2\tilde{\Delta} \sigma^z B^\dagger B + \sum_{kj} H_{kj} a^\dagger_k a_j
+ \tilde{\Delta}[(\sigma^x + i \sigma^y)B + (\sigma^x - i\sigma^y)B^\dagger]$$
or equivalently
$$H_{polRWA} =\frac{\tilde\Delta}{2}\sigma^z(1 - 4 B^\dagger B) + \sum_{kj} H_{kj} a^\dagger_k a_j
+ 2\tilde{\Delta}[\sigma^+ B + \sigma^- B^\dagger]$$

Note also that the polaron representation is *rotated* with respect to the lab frame. If we wish to estimate the lab expectation value of $\sigma_z$ we have to compute $\frac{\tilde\Delta}{\Delta}\sigma^z(1-4B^\dagger B)$ instead!

The following class implements both $H_{RWA}$ and $H_{polRWA}$. Note that they are structurally very similar. The only differences are the lack of the $B^\dagger B$ correction and the fact that the polRWA model uses couplings $2\tilde{\Delta}f_k$ instead of the original $g_k$.

In [ ]:
# file: seeq/models/spinboson.py
from seeq.evolution import evolve

class SinglePhotonHamiltonian(LinearOperator):
    """Class for the single-photon limit of the Polaron or RWA
    Hamiltonians of the spin-boson model.
    
    Parameters
    ----------
    lattice  -- an object of Lattice class representing the bosonic environment
    r        -- position of the quantum emitter in `lattice`
    g        -- vector of couplings between the emitter and the environment or
                a scalar to multiply the output of `lattice.coupling_at(r)`.
    Δ        -- energy gap of the bare emitter (defaults to 1.)
    polaron  -- True if you wish to use the polaron approximation.
    Lanczos  -- True if you wish to simplify the lattice using the Lanczos method
    nmodes   -- number of modes to keep in the Lanczos method
    """

    def __init__(self, lattice, g=1., r=None, Δ=1.0, polaron=False,
                 Lanczos=False, nmodes=None):
        #
        # `N` is the number of photon modes. The Hilbert space of
        # photons include 1 photon state per mode plus one state for
        # the excited qubit, hence dimension is N+1
        N = lattice.size
        L = N+1
        super(SinglePhotonHamiltonian, self).__init__(np.complex128, (L,L))
        self.N = N
        self.size = L

        if np.isscalar(g) and r is None:
            raise Exception('You need to provide the position of the emitter.')
        self.g = g = g * lattice.coupling_at(r)
        self.Δ = Δ
        #
        # When using the Lanczos optimization, the vector of couplings
        # changes, because it is expressed in a new basis.
        self.Lanczos = Lanczos
        if Lanczos:
            self.old_lattice = lattice
            lattice = lattice = LanczosEnv(lattice, g, nmodes=nmodes)
            g = lattice.g
        self.lattice = lattice
        #
        # When using the polaron approximation, we compute the displacements
        # the renormalized frequency and the effective couplings
        self.polaron = polaron
        if polaron:
            f, Δr = SilbeyHarris(lattice, Δ=Δ)
            self.f = f
        #
        # The qubit operator is excited only in the state '0'
        sz = sp.diags([[1.0]+[-1.0]*N], offsets=[0], format='coo')
        #
        # Boson Hamiltonian
        Hw = sp.bmat([[0.0, sp.coo_matrix((1,N))],
                      [sp.coo_matrix((N,1)), lattice.hamiltonian()]])
        #
        # Number of excitations in photon space
        nphotons = sp.diags([[0.0]+[1.0]*N], offsets=[0], format='coo')
        #
        # Given a vector `g` construct `\sum_k g_k \sigma^+ a_k`
        spB = sp.bmat([[0.0, g.conj()],
                       [sp.coo_matrix((N,1)), sp.coo_matrix((N,N))]])
        smBd = spB.T.conj()
        #
        # These are the differences between the pure polaron and
        # the RWA model. The polaron model involves a dense matrix
        # B^\dagger * B, which is why we code it as a LinearOperator
        # that does not need to build the matrix
        if polaron:
            Hint = 2.0 * DeltaR * (spB + smBd)
            polSz = lambda ψ: (Δr/Delta)* (sz @ (ψ - 4.0 * (smBd @ (spB @ ψ))))
            applyH = lambda ψ: Hw @ ψ + Hint @ ψ + 0.5 * Δ * polSz(ψ)
            self._applyH = _applyH
            self._H = LinearOperator(matvec=applyH, shape=(L,L))
            self.lab_sz = sp.linalg.LinearOperator(matvec=polSz, shape=(L,L))
        else:
            H = Hw + (spB + smBd) + 0.5 * Δ * sz
            self._applyH = lambda ψ: H @ ψ
            self.lab_sz = sz
        self.sz = sz
        self.nphotons = nphotons

    def Hamiltonian(self):
        return self._H
    
    def _matvec(self, ψ):
        return self._applyH(ψ)
    
    def _matmat(self, A):
        return self._applyH(A)

    def one_photon(self, ϕ):
        """Return a state with a photon wavepacket.
        
        Parameters
        ----------
        ϕ  -- Photon wavepacket. Complex vector of size N, with
              the amplitudes of the state on each lattice site."""
        return np.concatenate([[0.], ϕ / np.linalg.norm(ϕ)])

    def excited(self):
        """Return a state with an excited qubit."""
        return np.concatenate([[1.], np.zeros(self.N,)])

    def photon_part(self, ψ):
        if self.Lanczos:
            return self.lattice.basis @ ψ[1:]
        else:
            return ψ[1:]

    def emission(self, T, steps=20, collect_states=False, **kwdargs):
        """Simulate the spontaneous emission of the excited impurity on
        a background of no photons.
        
        Parameters
        ----------
        T, steps       -- Similar semantics to seeq.evolution.evolve()
        collect_states -- True if we collect all states
        **kwdargs      -- All other arguments accepted by evolve()
        
        Returns
        -------
        t              -- Vector of times
        P1             -- Excited population of the qubit
        Pphoton        -- Density of photons
        ψ              -- Matrix of all states (if collect_states) or
                          final state otherwise"""
        if np.isscalar(T):
            T = np.linspace(0, T, steps)
        times = []
        out_ψ = []
        P1 = []
        Pphoton = []
        for (t, ψt) in evolve(self.excited(), self, T, constant=True, **kwdargs):
            if collect_states:
                out_ψ.append(ψt)
            else:
                out_ψ = ψt
            times.append(t)
            P1.append(np.abs(ψt[0])**2)
            Pphoton.append(np.abs(ψt[1:])**2)
        return np.array(times), np.array(P1), np.array(Pphoton), np.array(out_ψ)

## Applications

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

### a) Chain mapping lattice

In [ ]:
from seeq.models.lattice import SquareLattice
from seeq.models.lattice_plot import plot_lattice, plot_field2d

def show_Lanczos_modes(L=20):
    #
    # Create a 1D lattice with hopping `J` and on-site frequency `ω`
    lattice = SquareLattice(L, Jx=1., ω=1.)
    #
    # Emitter position
    r = (int(L/2),int(L/2),0)
    #
    # Lanczos environment
    lanczos = LanczosEnv(lattice, lattice.coupling_at(r))
    
    fig, ((ax1, ax2),(ax3,ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10,10))
    plot_lattice(lattice, ax=ax1)
    plot_field2d(lattice, np.abs(lanczos.basis[:,0]), ax=ax1, cmap='Oranges')
    plot_lattice(lattice, ax=ax2)
    plot_field2d(lattice, np.abs(lanczos.basis[:,1]), ax=ax2, cmap='Oranges')
    plot_lattice(lattice, ax=ax3)
    plot_field2d(lattice, np.abs(lanczos.basis[:,4]), ax=ax3, cmap='Oranges')
    plot_lattice(lattice, ax=ax4)
    plot_field2d(lattice, np.abs(lanczos.basis[:,8]), ax=ax4, cmap='Oranges')
    plt.tight_layout()

show_Lanczos_modes()

### b) Spontaneous emission in 1D

Note how the class implements a method, `emission()`, that simulates the decay of the quantum emitter over a given period of time.

In [ ]:
from seeq.models.lattice import Lattice1D

def test_emission(Δ=2., x=0, g=0.05, L=100, J=1., ω=2., T=50, steps=50):
    #
    # Create a 1D lattice with hopping `J` and on-site frequency `ω`
    lattice = Lattice1D(L, J=J, ω=ω)
    #
    # Emitter position
    r = (x,0,0)
    #
    # Create a spin-boson model
    SB = SinglePhotonHamiltonian(lattice, Δ=Δ, g=g, r=r)
    
    t, P1, Pphoton, _ = SB.emission(T, steps=steps)
    
    fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(10,4))
    ax1.plot(t * J, P1)
    ax1.set_xlabel('$t J$')
    ax1.set_ylabel('$P_e$')
    X, Y = np.meshgrid(np.arange(lattice.size), t*J)
    ax2.pcolormesh(X, Y, Pphoton)
    ax2.set_xlabel('$x$')
    ax2.set_ylabel('$t J$')

test_emission()

Using this utility, we can study the spontaneous emission of a two-level system as a function of time, and how the created photon wavepacket propagates through the lattice.